In [62]:
def extract_sentences(text):
    return [line.strip()+' <EOS>' for line in text.split(".") if len(line.split(" ")) > 2]

In [104]:
class BytePairEncoder:
    def __init__(self, vocab_size=1000):
        
        self.vocab_size = vocab_size
        self.vocab = {}  
        self.inverse_vocab = {} 
        self.merges = []  

    def train(self, content):
        texts = extract_sentences(content)
        #word_counts = {low:1, lower:2, high:1}
        word_counts = {}
        for text in texts:
            idx = 0
            for word in text.split():
                if idx!=0:
                    word = "Ġ"+word
                else:
                    idx += 1
                if word not in word_counts:
                    word_counts[word] = 0
                word_counts[word] += 1

        #word_vocab = {l o w:1, l o w e r:2, h i g h:1}
        word_vocab = {}

        for word, count in word_counts.items():
            if word == 'Ġ<EOS>':
                word_vocab['Ġ' + ' <EOS>'] = count  
        
            parts = word.split('Ġ')
            core_word = parts[-1]  
        
            if re.match(r'^[\d,.-]+$', core_word):  
                word_vocab['Ġ ' + core_word] = count  
        
            elif re.search(r'[க-ஹ]+[a-zA-Z]+|[a-zA-Z]+[க-ஹ]+', core_word):  
                word_vocab['Ġ ' + core_word] = count  
        
            elif re.match(r'^[\W_]+$', core_word):  
                word_vocab['Ġ ' + core_word] = count  
        
            else:
                chars = tamil.utf8.get_letters(word) 
                word_vocab[" ".join(chars)] = count 

        #initial_vocab = {l, o ,w, e, r, h, i, g}
        initial_vocab = set()
        for word in word_vocab.keys():
            initial_vocab.update(word.split())

        #'b': 0, # 'd': 1}, self.vocab[char]
        #{0: 'b',1: 'd'} self.inverse_vocab[i]
        for i, char in enumerate(sorted(initial_vocab)):
            self.vocab[char] = i
            self.inverse_vocab[i] = char
        
        #next_id = 12 for example
        next_id = len(self.vocab)
        print(next_id)
        
        while len(self.vocab) < self.vocab_size:
            pairs = self._get_pairs(word_vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            new_token = best_pair[0] + best_pair[1]
            print(best_pair)
            self.merges.append(best_pair)
            
            self.vocab[new_token] = next_id
            self.inverse_vocab[next_id] = new_token
            next_id += 1
            
            new_vocab = {}
            bigram = ' '.join(best_pair)
            replacement = ''.join(best_pair)
            
            for word, count in word_vocab.items():
                new_word = word.replace(bigram, replacement)
                new_vocab[new_word] = count
                
            word_vocab = new_vocab
            
            if len(self.vocab) >= self.vocab_size:
                l = len(self.vocab)
                self.vocab['<UNK>'] = len(l)
                self.inverse_vocab[l] = '<UNK>'
                break

        
    
    def _get_pairs(self, word_vocab):
        pairs = {}
        for word, count in word_vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                pairs[pair] = pairs.get(pair, 0) + count
        return pairs
    
    def encode(self, text):
        tokens = []
        words = text.split()
        idx = 0
        for word in text.split():
            if idx!=0:
                word_to_encode = "Ġ"+word
            else:
                word_to_encode = word
                idx += 1
            
            if re.match(r'^[\d,.-]+$', word):  
                if i != 0:
                    tokens.append(self.vocab.get('Ġ', self.vocab.get(' ')))
                tokens.extend([self.vocab.get(char, self._get_unknown_token_id()) for char in word])
                continue
            elif re.search(r'[க-ஹ]+[a-zA-Z]+|[a-zA-Z]+[க-ஹ]+', word):  # Mixed scripts
                if i != 0:
                    tokens.append(self.vocab.get('Ġ', self.vocab.get(' ')))
                tokens.extend([self.vocab.get(char, self._get_unknown_token_id()) for char in word])
                continue
            elif re.match(r'^[\W_]+$', word):  # Non-word characters
                if i != 0:
                    tokens.append(self.vocab.get('Ġ', self.vocab.get(' ')))
                tokens.extend([self.vocab.get(char, self._get_unknown_token_id()) for char in word])
                continue
            
            chars = tamil.utf8.get_letters(word_to_encode)
            word_tokens = " ".join(chars)
            
            for pair in self.merges:
                bigram = ' '.join(pair)
                replacement = ''.join(pair)
                word_tokens = word_tokens.replace(bigram, replacement)
            
            for subword in word_tokens.split():
                if subword in self.vocab:
                    tokens.append(self.vocab[subword])
                else:
                    for char in subword:
                        if char in self.vocab:
                            tokens.append(self.vocab[char])
                        else:
                            tokens.append(self._get_unknown_token_id())
        
        return tokens

    def _get_unknown_token_id(self):
        return self.vocab.get('<UNK>', len(self.vocab))
    
    def decode(self, ids):
        tokens = [self.inverse_vocab.get(id, '<UNK>') for id in ids]
        
        text = ""
        for token in tokens:
            if token.startswith('Ġ'):
                text += ' ' + token[1:]
            else:
                text += token
        
        text = text.strip()
        return text



In [105]:
# Example usage
if __name__ == "__main__":
    # Sample training data
    content = '''
    this is a test. இத்திரைப்படம் 22 பெப்ரவரி 2019 அன்று திரைக்கு வந்தது. இத்திரைப்படத்தை லியோன் ஜேம்ஸ் இயக்கியுள்ளார். இத்திரைப்படத்தை வேல் பிலிம்ஸ் இன்டர்நேஷனல் தயாரித்துள்ளனர். இத்திரைப்படத்தில் நாஞ்சில் சம்பத், ஆர். கே. ரித்திஷ், மயில்சாமி ஆகியோரும் நடித்துள்ளனர். லால்குடி கருப்பையா காந்தி என்பதன் சுருக்கமே எல். கே. ஜி என்பதாகக் கூறப்படுகிறது. திரைப்படம் முழுவதும் சமகால அரசியல் சூழலை பகடி செய்யும் விதத்தில் அமைந்துள்ளதாகவும், அதே நேரத்தில் அரசியல் விழிப்புணர்வை ஏற்படுத்தும் நகைச்சுவைத் திரைப்படமாகவும் வந்துள்ளதாக விமர்சனங்கள் தெரிவிக்கின்றன.
    '''
    content = '''
    this is இத்திரைப்படம். 
    '''
    # texts = extract_sentences(content)
    # print(texts)
    
    # texts = [
    #     "low lower lowest",
    #     "newer new newest lower",
    #     "wider width widest",
    #     "better best"
    # ]
    
    # Create and train the BPE model
    bpe = BytePairEncoder(vocab_size=120)
    bpe.train(content)
    
    # Print the vocabulary and merges
    print("Vocabulary:")
    for token, id in sorted(bpe.vocab.items(), key=lambda x: x[1]):
        print(f"{id}: '{token}'")
    
    print("\nMerges:")
    for i, merge in enumerate(bpe.merges):
        print(f"{i+1}: {merge[0]} + {merge[1]} -> {merge[0] + merge[1]}")
    
    # Test encoding and decoding
    test_text = "என்ன பண்ணனும்"
    encoded = bpe.encode(test_text)
    decoded = bpe.decode(encoded)
    
    print(f"\nOriginal: {test_text}")
    print(f"Encoded: {encoded}")
    print(f"Decoded: {decoded}")

    # word_counts = {}
    # for text in texts:
    #     idx = 0
    #     for word in text.split():
    #             if idx!=0:
    #                 word = "Ġ"+word
    #             else:
    #                 idx += 1
    #             if word not in word_counts:
    #                 word_counts[word] = 0
    #             word_counts[word] += 1

19
('i', 's')
('<', 'E')
('<E', 'O')
('<EO', 'S')
('<EOS', '>')
('t', 'h')
('th', 'is')
('Ġ', 'is')
('Ġ', 'இ')
('Ġஇ', 'த்')
('Ġஇத்', 'தி')
('Ġஇத்தி', 'ரை')
('Ġஇத்திரை', 'ப்')
('Ġஇத்திரைப்', 'ப')
('Ġஇத்திரைப்ப', 'ட')
('Ġஇத்திரைப்பட', 'ம்')
('Ġ', '<EOS>')
Vocabulary:
0: '<'
2: '>'
3: 'E'
4: 'O'
5: 'S'
6: 'h'
7: 'i'
8: 's'
9: 't'
10: 'Ġ'
11: 'இ'
12: 'ட'
13: 'தி'
14: 'த்'
15: 'ப'
16: 'ப்'
17: 'ம்'
18: 'ரை'
19: 'is'
20: '<E'
21: '<EO'
22: '<EOS'
23: '<EOS>'
24: 'th'
25: 'this'
26: 'Ġis'
27: 'Ġஇ'
28: 'Ġஇத்'
29: 'Ġஇத்தி'
30: 'Ġஇத்திரை'
31: 'Ġஇத்திரைப்'
32: 'Ġஇத்திரைப்ப'
33: 'Ġஇத்திரைப்பட'
34: 'Ġஇத்திரைப்படம்'
35: 'Ġ<EOS>'

Merges:
1: i + s -> is
2: < + E -> <E
3: <E + O -> <EO
4: <EO + S -> <EOS
5: <EOS + > -> <EOS>
6: t + h -> th
7: th + is -> this
8: Ġ + is -> Ġis
9: Ġ + இ -> Ġஇ
10: Ġஇ + த் -> Ġஇத்
11: Ġஇத் + தி -> Ġஇத்தி
12: Ġஇத்தி + ரை -> Ġஇத்திரை
13: Ġஇத்திரை + ப் -> Ġஇத்திரைப்
14: Ġஇத்திரைப் + ப -> Ġஇத்திரைப்ப
15: Ġஇத்திரைப்ப + ட -> Ġஇத்திரைப்பட
16: Ġஇத்திரைப்பட + ம் -> Ġஇத்திரைப்படம்
1

In [103]:
bpe.vocab

{'<': 0,
 '<EOS>': 1,
 '>': 2,
 'E': 3,
 'O': 4,
 'S': 5,
 'h': 6,
 'i': 7,
 's': 8,
 't': 9,
 'Ġ': 10,
 'இ': 11,
 'ட': 12,
 'தி': 13,
 'த்': 14,
 'ப': 15,
 'ப்': 16,
 'ம்': 17,
 'ரை': 18,
 'is': 996985}

In [83]:
import re
import tamil

vocab = {}

for word, count in word_counts.items():
    if word == 'Ġ<EOS>':
        vocab['Ġ' + ' <EOS>'] = count  

    parts = word.split('Ġ')
    core_word = parts[-1]  

    if re.match(r'^[\d,.-]+$', core_word):  
        vocab['Ġ ' + core_word] = count  

    elif re.search(r'[க-ஹ]+[a-zA-Z]+|[a-zA-Z]+[க-ஹ]+', core_word):  
        vocab['Ġ ' + core_word] = count  

    elif re.match(r'^[\W_]+$', core_word):  
        vocab['Ġ ' + core_word] = count  

    else:
        chars = tamil.utf8.get_letters(word) 
        vocab[" ".join(chars)] = count  



In [84]:
vocab

{'t h i s': 1,
 'Ġ i s': 1,
 'Ġ a': 1,
 'Ġ t e s t': 1,
 'Ġ <EOS>': 9,
 'Ġ < E O S >': 9,
 'இ த் தி ரை ப் ப ட ம்': 1,
 'Ġ 22': 1,
 'Ġ பெ ப் ர வ ரி': 1,
 'Ġ 2019': 1,
 'Ġ அ ன் று': 1,
 'Ġ தி ரை க் கு': 1,
 'Ġ வ ந் த து': 1,
 'இ த் தி ரை ப் ப ட த் தை': 2,
 'Ġ லி யோ ன்': 1,
 'Ġ ஜே ம் ஸ்': 1,
 'Ġ இ ய க் கி யு ள் ளா ர்': 1,
 'Ġ வே ல்': 1,
 'Ġ பி லி ம் ஸ்': 1,
 'Ġ இ ன் ட ர் நே ஷ ன ல்': 1,
 'Ġ த யா ரி த் து ள் ள ன ர்': 1,
 'இ த் தி ரை ப் ப ட த் தி ல்': 1,
 'Ġ நா ஞ் சி ல்': 1,
 'Ġ ச ம் ப த் ,': 1,
 'Ġ ஆ ர்': 1,
 'ரி த் தி ஷ் ,': 1,
 'Ġ ம யி ல் சா மி': 1,
 'Ġ ஆ கி யோ ரு ம்': 1,
 'Ġ ந டி த் து ள் ள ன ர்': 1,
 'லா ல் கு டி': 1,
 'Ġ க ரு ப் பை யா': 1,
 'Ġ கா ந் தி': 1,
 'Ġ எ ன் ப த ன்': 1,
 'Ġ சு ரு க் க மே': 1,
 'Ġ எ ல்': 1,
 'ஜி': 1,
 'Ġ எ ன் ப தா க க்': 1,
 'Ġ கூ ற ப் ப டு கி ற து': 1,
 'தி ரை ப் ப ட ம்': 1,
 'Ġ மு ழு வ து ம்': 1,
 'Ġ ச ம கா ல': 1,
 'Ġ அ ர சி ய ல்': 2,
 'Ġ சூ ழ லை': 1,
 'Ġ ப க டி': 1,
 'Ġ செ ய் யு ம்': 1,
 'Ġ வி த த் தி ல்': 1,
 'Ġ அ மை ந் து ள் ள தா க வு ம் ,': 1,
 'Ġ அ தே': 1

In [45]:
import tamil

word = "தமிழ்"
syllables = tamil.utf8.get_letters(word)
print(syllables)


['த', 'மி', 'ழ்']


In [14]:
initial_vocab = set()
for word in vocab.keys():
    initial_vocab.update(word.split())


In [15]:
vocab, inverse_vocab, next_id, initial_vocab

({'b': 0,
  'd': 1,
  'e': 2,
  'h': 3,
  'i': 4,
  'l': 5,
  'n': 6,
  'o': 7,
  'r': 8,
  's': 9,
  't': 10,
  'w': 11},
 {0: 'b',
  1: 'd',
  2: 'e',
  3: 'h',
  4: 'i',
  5: 'l',
  6: 'n',
  7: 'o',
  8: 'r',
  9: 's',
  10: 't',
  11: 'w'},
 12,
 {'b', 'd', 'e', 'h', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w'})

In [ ]:
vocab_size=20
while len(vocab) < self.vocab_size:
            pairs = self._get_pairs(vocab)
            if not pairs:
                break
                
            best_pair = max(pairs, key=pairs.get)
            new_token = best_pair[0] + best_pair[1]
            
            # Add the merge to our list
            self.merges.append(best_pair)
            
            # Update vocabulary with the new token
            self.vocab[new_token] = next_id
            self.inverse_vocab[next_id] = new_token
            next_id += 1
            
            # Update the word counts with the new merged pair
            new_vocab = {}
            bigram = ' '.join(best_pair)
            replacement = ''.join(best_pair)
            
            for word, count in vocab.items():
                new_word = word.replace(bigram, replacement)
                new_vocab[new_word] = count
                
            vocab = new_vocab
            
            # Stop if we've reached the desired vocab size
            if len(self.vocab) >= self.vocab_size:
                break

In [13]:
a = "வணக்கம்"
[i for i in a]

['வ', 'ண', 'க', '்', 'க', 'ம', '்']

In [19]:
def _get_pairs(word_vocab):
        pairs = {}
        for word, count in word_vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                pairs[pair] = pairs.get(pair, 0) + count
        return pairs

In [22]:
_get_pairs({'l o w': 1})

{('l', 'o'): 1, ('o', 'w'): 1}